In [1]:
%pip install spacy nltk 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import spacy
import nltk
from nltk import CFG, ChartParser
from nltk.tokenize import word_tokenize


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/riyagandhi/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/riyagandhi/Library/Python/3.10/lib/python/site-packages/traitlets/c

In [3]:
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/riyagandhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/riyagandhi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
import pandas as pd 
VLMresponses = pd.read_csv("VLM_responses_Baseline_i.csv")
VLMresponses = VLMresponses[VLMresponses["Accurate"] != "FALSE"]
VLMresponses = VLMresponses.drop(["Why it Failed?", "Accurate"], axis='columns')
VLMresponses = VLMresponses.reset_index(drop=True)
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""inte..."
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",..."
2,I want to add a red bag to my bag.,"Based on the image, the user would need to fol...",NaN
3,I want to access my lessons offline.,"Based on the image, the user can complete the ...",NaN
4,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ..."


In [5]:
# Sample Input
text = VLMresponses.iloc[1]["AssistantResponse"]
print(text)

jsonForm = VLMresponses.iloc[1]["JSON"]
print(jsonForm)

To change the contact name to Jane Do using the current screenshot, the user would need to follow these steps:

1. Tap on the 'Barack Obama' text field to edit the name.
2. Type 'Jane Do' in the text field.
3. Tap on the 'Save' button to confirm the changes.

Please note that the actual number of steps might be different depending on the app's design and functionality.
{
   "task":"change contact name to Jane Do",
   "interactions":[
      "Save",
      "text"
   ]
}


In [6]:
doc = nlp(text)

In [7]:
from spacy import displacy
displacy.render(doc, style="dep")

In [8]:
def extract_task(doc):
    task = None
    for sent in doc.sents: # break down the document into sentences
        verbs = [token for token in sent if token.pos_ == "VERB"] # find all verbs in the sentence
        for verb in verbs: # look through every word inthe sentence
            verb_subtree = [t for t in verb.subtree if t.i >= verb.i] # find all verbs in the subtree of the current verb
            print(f"verb_subtree for {verb.text} with dep {verb.dep_}: {[t.text for t in verb_subtree]}") # print the subtree of the current verb
            if verb.dep_ in ["advcl"]:
                task = " ".join([t.text for t in verb_subtree]) #  subtree includes the verb and all words attached to it in the tree.
               
            if task: # if a task has been found, break out of the loop
                return task
    return task

print(extract_task(doc))

verb_subtree for change with dep advcl: ['change', 'the', 'contact', 'name', 'to', 'Jane']
change the contact name to Jane


In [9]:
def extract_buttons(doc):
    buttons = set()
    for sent in doc.sents:
        for token in sent:
            button_name = None
            # if (token.lemma_).lower() in ("tap", "press", "click"):
            #     # button_name = None
            #     for child in token.children:
            #         if child.dep_ in ("prep"):
            #             for grandchild in child.children:
            #                 for word in grandchild.subtree:
            #                     if word.text.lower() in ["button", "icon", "field"]:
            #                         break
            #                     if word.dep_ in ("compound", "amod", "nmod", "pobj", "npadvmod"):
            #                         print(f"Adding word to label: {word.text}")
            #                         if button_name is None:
            #                             button_name = word.text
            #                         else:
            #                             button_name += word.text
            #                         button_name += " "  # Add space between words
            #             buttons.add("".join(button_name.strip())) 
            if (token.lemma_).lower() in ("button", "icon", "field", "option", "section", "link"):
                button_phrase = []
                for child in token.children:
                    if child.dep_ in ("nmod", "compound", "amod", "attr", "appos", "poss"):
                        button_phrase.extend([t.text for t in child.subtree])
                        print(f"Adding button phrase {button_phrase}")
                if button_phrase:
                    if button_phrase[0].lower() in ("the", "a", "an"):
                        button_phrase = button_phrase[1:]
                    buttons.add(" ".join(button_phrase).replace("\"", "").replace("'", "").strip())
    return buttons

print(extract_buttons(doc))

Adding button phrase ['Barack', 'Obama', "'"]
Adding button phrase ['Barack', 'Obama', "'", 'text']
Adding button phrase ['text']
Adding button phrase ['Save']
{'text', 'Barack Obama  text', 'Save'}
